In [1]:
import re
from vnpy.trader.constant import Exchange
from vnpy.app.cta_strategy.base import EXCHANGE_SYMBOL_DICT
from datetime import datetime, timedelta

In [36]:
fromDate = datetime.strptime('2012-6-10', '%Y-%m-%d')
toDate = datetime.strptime('2017-12-01', '%Y-%m-%d')

count = 0
while fromDate.date() != toDate.date():
    count += 1
    fromDate = fromDate + timedelta(1)

print(count)

2000


In [16]:
# 当前时间戳
import time
a = int(time.time())
print(a)

1563864223


In [59]:
# 时间戳转时间
timeStamp = 1512136800
date = datetime.fromtimestamp(timeStamp)
print(date)

2017-12-01 22:00:00


In [23]:
# 时间转时间戳
a = '2019-07-20 08:00:00'
b = datetime.strptime(a, '%Y-%m-%d %H:%M:%S')
print(b.timestamp())

1563580800.0


In [3]:
import requests
URL_EXCHANGES = 'https://www.cryptocompare.com/api/data/exchanges'
response = requests.get(URL_EXCHANGES).json()
data = response['Data']
print(data.keys())

dict_keys(['decoin', 'Binance', 'Bitstamp', 'BitTrex', 'OKCoin', 'BTER', 'Kraken', 'Cexio', 'Yacuna', 'Coinbase', 'LocalBitcoins', 'itBit', 'HitBTC', 'BTC38', 'Coinfloor', 'BTCExchange', 'hbus', 'LakeBTC', 'Bit2C', 'CCEX', 'Gatecoin', 'Yunbi', 'WavesDEX', 'CCEDK', 'Cryptopia', 'Exmo', 'Yobit', 'BitBay', 'Korbit', 'BTCMarkets', 'Coincheck', 'QuadrigaCX', 'BitSquare', 'Vaultoro', 'MercadoBitcoin', 'Unocoin', 'Bitso', 'BTCXIndia', 'Paymium', 'TheRockTrading', 'bitFlyer', 'Quoine', 'Liqui', 'Luno', 'CryptoX', 'EtherDelta', 'TuxExchange', 'CHBTC', 'bitFlyerFX', 'LiveCoin', 'Tidex', 'CoinCorner', 'BitMarket', 'Coinone', 'Bleutrade', 'EthexIndia', 'Surbitcoin', 'Bithumb', 'Foxbit', 'Zaif', 'Jubi', 'ViaBTC', 'Novaexchange', 'Lykke', 'Remitano', 'Coinroom', 'Abucoins', 'BXinth', 'HuobiPro', 'OKEX', 'Gateio', 'TrustDEX', 'EXX', 'Kucoin', 'BitGrail', 'BitZ', 'BitFlip', 'TradeSatoshi', 'Coincap', 'ChileBit', 'VBTC', 'Velox', 'Graviex', 'ExtStock', 'BitBank', 'CoinDeal', 'AidosMarket', 'TokenStore'

In [10]:
from pymongo import MongoClient, ASCENDING, DESCENDING
from vnpy.app.cta_strategy.base import TICK_DB_NAME, MINUTE_DB_NAME, DAILY_DB_NAME, MinuteDataBaseName, HourDataBaseName
from datetime import datetime
from vnpy.trader.object import BarData, TickData

In [73]:
# ====== Tick ======
client = MongoClient('localhost', 27017, serverSelectionTimeoutMS = 600)       # Mongo连接
client.server_info()
dbTick = client[TICK_DB_NAME]
symbol_list = ['BTCUSD', 'ETHUSD', 'EOSUSD']
for symbol in symbol_list:
    cl = dbTick[symbol]
    cl.create_index('datetime')         # 添加索引
    start = datetime.strptime('2019-11-18 07:59:50', '%Y-%m-%d %H:%M:%S')
    end = datetime.strptime('2019-11-19 08:00:10', '%Y-%m-%d %H:%M:%S')
    flt = {'datetime': {'$gte':start,
                        '$lte':end}}
    cursor = cl.find({}).sort('datetime', DESCENDING)
    count = 0
    for dic in cursor:
        tick = TickData(gateway_name='', symbol='', exchange=None, datetime=None)
        tick.__dict__ = dic
        print(f'vt_symbol:{tick.vt_symbol}\t{tick.datetime}\t{tick.symbol}\t{tick.last_price}')
        count += 1
        if count >= 1:
            break

vt_symbol:BTCUSD.BYBIT	2020-03-25 07:52:46.091000	BTCUSD	6754.5
vt_symbol:ETHUSD.BYBIT	2020-03-25 08:01:59.786000	ETHUSD	137.85
vt_symbol:EOSUSD.BYBIT	2020-03-25 08:01:59.204000	EOSUSD	2.336


In [37]:
# ====== Minute ======
client = MongoClient('localhost', 27017)
db = client[MINUTE_DB_NAME]
#collection = db['BTCUSD.BYBIT']
collection = db['ZN2006']
start = datetime.strptime('2020-3-24 00:00:00', '%Y-%m-%d %H:%M:%S')
end = datetime.strptime('2020-3-25 00:00:00', '%Y-%m-%d %H:%M:%S')
flt = {'datetime': {'$gte':start,
                    '$lte':end}}
cursor = collection.find().sort('datetime', DESCENDING)
count = 0
for dic in cursor:
    print(dic['datetime'], dic['open_price'])
    count += 1
    if count >= 6:
        break

2020-04-08 15:00:00 15675.0
2020-04-08 14:59:00 15680.0
2020-04-08 14:58:00 15685.0
2020-04-08 14:57:00 15690.0
2020-04-08 14:56:00 15690.0
2020-04-08 14:55:00 15690.0


In [19]:
# ====== Daily ======
client = MongoClient('localhost', 27017)
db = client[DAILY_DB_NAME]
symbol_list = ['BTCUSD.BYBIT', 'ETHUSD.BYBIT', 'EOSUSD.BYBIT']
#symbol_list = ['BTC.USDT.OKEX', 'ETH.USDT.OKEX', 'EOS.USDT.OKEX']
#symbol_list = ['OKEX/BTC.USDT', 'OKEX/ETH.USDT', 'OKEX/EOS.USDT']
#symbol_list = ['Poloniex/BTC.USDT', 'Poloniex/ETH.USDT', 'Poloniex/EOS.USDT']
symbol_list = ['BTCUSD.BYBIT']
for symbol in symbol_list:
    collection = db[symbol]
    start = datetime.strptime('2019-11-27 00:00:00', '%Y-%m-%d %H:%M:%S')
    end = datetime.strptime('2019-11-27 00:06:00', '%Y-%m-%d %H:%M:%S')
    flt = {'datetime': {'$gte':start,
                        '$lte':end}}
    cursor = collection.find({}).sort('datetime', DESCENDING)
    count = 0
    for dic in cursor:
        symbol = dic['symbol']
        the_datetime = dic['datetime']
        open_price = dic['open_price']
        high_price = dic['high_price']
        low_price = dic['low_price']
        close_price = dic['close_price']
        print(f'{symbol}\t{the_datetime}\t{open_price}\t{high_price}\t{low_price}\t{close_price}')
        count += 1
        if count > 60:
            break

BTCUSD.BYBIT	2020-05-12 08:00:00	8567.5	8983.0	8537.0	8818.5
BTCUSD.BYBIT	2020-05-11 08:00:00	8724.5	9170.0	8165.0	8567.5
BTCUSD.BYBIT	2020-05-10 08:00:00	9550.5	9585.0	8088.0	8724.5
BTCUSD.BYBIT	2020-05-09 08:00:00	9813.0	9919.5	9532.0	9550.5
BTCUSD.BYBIT	2020-05-08 08:00:00	10010.0	10057.0	9718.0	9813.0
BTCUSD.BYBIT	2020-05-07 08:00:00	9151.0	10098.5	9025.5	10010.0
BTCUSD.BYBIT	2020-05-06 08:00:00	9021.5	9416.5	8916.0	9151.0
BTCUSD.BYBIT	2020-05-05 08:00:00	8883.0	9120.0	8764.0	9021.5
BTCUSD.BYBIT	2020-05-04 08:00:00	8905.0	8972.0	8529.0	8883.0
BTCUSD.BYBIT	2020-05-03 08:00:00	8982.0	9205.5	8728.0	8905.0
BTCUSD.BYBIT	2020-05-02 08:00:00	8831.5	9022.0	8756.5	8982.0
BTCUSD.BYBIT	2020-05-01 08:00:00	8628.5	9073.0	8623.0	8831.5
BTCUSD.BYBIT	2020-04-30 08:00:00	8789.0	9482.5	8409.0	8628.5
BTCUSD.BYBIT	2020-04-29 08:00:00	7749.5	8974.0	7718.5	8789.0
BTCUSD.BYBIT	2020-04-28 08:00:00	7786.5	7796.5	7663.5	7749.5
BTCUSD.BYBIT	2020-04-27 08:00:00	7703.0	7813.0	7629.0	7786.5
BTCUSD.BYBIT	2020-04

In [5]:
symbol_list = ['RB2005', 'HC2005', 'SM2005', 'J2005', 'ZC2005', 'TA2005',
              'RB99', 'HC99', 'SM99', 'J99', 'ZC99', 'TA99']
symbol_list = ['BTCUSD.BYBIT']
client = MongoClient('localhost', 27017)
db = client[DAILY_DB_NAME]
for symbol in symbol_list:
    collection = db[symbol]
    cursor = collection.find({}).sort('datetime', DESCENDING)
    count = 0
    for dic in cursor:
        count += 1
        if count == 1:
            symbol = dic['symbol']
            exchange = dic['exchange']
            the_datetime = dic['datetime']
            open_price = dic['open_price']
            high_price = dic['high_price']
            low_price = dic['low_price']
            close_price = dic['close_price']
            print(f'{symbol}\t{exchange}\t{the_datetime}\t{open_price}\t{high_price}\t{low_price}\t{close_price}')
            break
        
    print('\n')

BTCUSD.BYBIT	None	2019-12-24 08:00:00	7310.0	7429.0	7155.5	7254.0




In [59]:
# ====== 检查Tick质量 ======
client = MongoClient('localhost', 27017, serverSelectionTimeoutMS = 600)       # Mongo连接
client.server_info()
dbTick = client[TICK_DB_NAME]
cl = dbTick['BTCUSD']
cl.create_index('datetime')         # 添加索引
start = datetime.strptime('2019-12-20 07:59:50', '%Y-%m-%d %H:%M:%S')
end = datetime.strptime('2019-12-30 08:00:10', '%Y-%m-%d %H:%M:%S')
flt = {'datetime': {'$gte':start,
                    '$lte':end}}
cursor = cl.find(flt).sort('datetime')
count = 0
for dic in cursor:
    tick = TickData(gateway_name='', symbol='', exchange=None, datetime=None)
    tick.__dict__ = dic
    if tick.last_price == 0:
        print(f'vt_symbol:{tick.vt_symbol}\t{tick.datetime}\t{tick.symbol}\t{tick.last_price}\t{tick.ask_price_1}\t{tick.bid_price_1}')
    count += 1
print(count)

IndentationError: expected an indented block (<ipython-input-59-025e50492adc>, line 18)

In [4]:
key = '黄金时代'
msg = f"{key} 辉煌"
msg += f"\t{key} 盛世"
print(msg)

黄金时代 辉煌	黄金时代 盛世


In [9]:
from copy import copy
a = [[1, 2], [3, 4], [5, 6]]
b = [[1, 2], [3, 4], [5, 6]]
c = copy(a)
c += b
print(a)
print(b)
print(c)
temp = [6, 6]
c.append(temp)
print(c)

[[1, 2], [3, 4], [5, 6]]
[[1, 2], [3, 4], [5, 6]]
[[1, 2], [3, 4], [5, 6], [1, 2], [3, 4], [5, 6]]
[[1, 2], [3, 4], [5, 6], [1, 2], [3, 4], [5, 6], [6, 6]]


In [14]:
a = None
b = None
if a == b:
    print('yes')

yes
